In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import files

In [ ]:
uploaded = files.upload()

Saving blogs.csv to blogs.csv


In [ ]:
df = pd.read_csv('blogs.csv')

In [ ]:
df.head()

,Data,Labels
0,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
1,Newsgroups: alt.atheism\nPath: cantaloupe.srv....,alt.atheism
2,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,alt.atheism
3,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
4,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...,alt.atheism


In [ ]:
print("\nChecking for missing values:")
print(df.isnull().sum())

print("\nDistribution of categories:")
print(df['Labels'].value_counts())

print("\nShape of the dataset:")
print(df.shape)



Checking for missing values:
Data      0
Labels    0
dtype: int64

Distribution of categories:
Labels
alt.atheism                 100
comp.graphics               100
talk.politics.misc          100
talk.politics.mideast       100
talk.politics.guns          100
soc.religion.christian      100
sci.space                   100
sci.med                     100
sci.electronics             100
sci.crypt                   100
rec.sport.hockey            100
rec.sport.baseball          100
rec.motorcycles             100
rec.autos                   100
misc.forsale                100
comp.windows.x              100
comp.sys.mac.hardware       100
comp.sys.ibm.pc.hardware    100
comp.os.ms-windows.misc     100
talk.religion.misc          100
Name: count, dtype: int64

Shape of the dataset:
(2000, 2)


In [ ]:
# Install spaCy
!pip install spacy

# Download the English model for spaCy
!python -m spacy download en_core_web_sm

import spacy
import string

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Define a function to preprocess the text
def preprocess_text_spacy(text):
    # Process the text using spaCy
    doc = nlp(text)

    # Tokenize and remove stopwords and punctuation, then convert to lowercase
    tokens = [token.text.lower() for token in doc if token.text not in string.punctuation and not token.is_stop]

    # Join tokens back into a single string
    cleaned_text = ' '.join(tokens)
    return cleaned_text

# Apply preprocessing to the 'Data' column
df['Cleaned_Data'] = df['Data'].apply(preprocess_text_spacy)

# Display the first few rows to confirm changes
print("First few rows after preprocessing:")
print(df[['Data', 'Cleaned_Data']].head())


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 89.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
First few rows after preprocessing:
                                                Data  \
0  Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...   
1  Newsgroups: alt.atheism\nPath: cantaloupe.srv....   
2  Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...   
3  Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...   
4  Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...   

                                        Cleaned_Data  
0  path cantaloupe.srv.cs.cmu.edu!magnesium.club....  
1  newsgroups alt.atheism \n path cantaloupe.srv....  
2  path cantaloupe.srv.cs.cmu.edu!das news.ha

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Cleaned_Data'])
print(f"TF-IDF matrix shape: {X.shape}")
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Labels'])
print("Encoded labels:", label_encoder.classes_)


TF-IDF matrix shape: (2000, 50875)
Encoded labels: ['alt.atheism' 'comp.graphics' 'comp.os.ms-windows.misc'
 'comp.sys.ibm.pc.hardware' 'comp.sys.mac.hardware' 'comp.windows.x'
 'misc.forsale' 'rec.autos' 'rec.motorcycles' 'rec.sport.baseball'
 'rec.sport.hockey' 'sci.crypt' 'sci.electronics' 'sci.med' 'sci.space'
 'soc.religion.christian' 'talk.politics.guns' 'talk.politics.mideast'
 'talk.politics.misc' 'talk.religion.misc']


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)
y_pred = nb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


Accuracy: 0.7900

Classification Report:
                          precision    recall  f1-score   support

             alt.atheism       0.60      0.83      0.70        18
           comp.graphics       0.72      0.72      0.72        18
 comp.os.ms-windows.misc       0.75      0.95      0.84        22
comp.sys.ibm.pc.hardware       0.72      0.84      0.78        25
   comp.sys.mac.hardware       0.88      0.67      0.76        21
          comp.windows.x       1.00      0.28      0.44        25
            misc.forsale       0.78      0.78      0.78        18
               rec.autos       0.81      0.94      0.87        18
         rec.motorcycles       0.82      0.88      0.85        16
      rec.sport.baseball       0.83      0.83      0.83        18
        rec.sport.hockey       0.65      1.00      0.79        15
               sci.crypt       0.70      1.00      0.83        19
         sci.electronics       0.69      0.56      0.62        16
                 sci.med       0.8

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd

nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

def get_sentiment(score):
    if score > 0.05:
        return 'positive'
    elif score < -0.05:
        return 'negative'
    else:
        return 'neutral'

df['Sentiment_Score'] = df['Cleaned_Data'].apply(lambda x: sia.polarity_scores(x)['compound'])
df['Sentiment'] = df['Sentiment_Score'].apply(get_sentiment)

print("First few rows with sentiment analysis:")
print(df[['Data', 'Sentiment_Score', 'Sentiment']].head())

sentiment_distribution = pd.crosstab(df['Labels'], df['Sentiment'], margins=True, margins_name="Total")
print("\nSentiment distribution across categories:")
print(sentiment_distribution)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


First few rows with sentiment analysis:
                                                Data  Sentiment_Score  \
0  Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...          -0.9785   
1  Newsgroups: alt.atheism\nPath: cantaloupe.srv....           0.8710   
2  Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...          -0.9939   
3  Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...          -0.9997   
4  Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...           0.9737   

  Sentiment  
0  negative  
1  positive  
2  negative  
3  negative  
4  positive  

Sentiment distribution across categories:
Sentiment                 negative  neutral  positive  Total
Labels                                                      
alt.atheism                     37        1        62    100
comp.graphics                   13        2        85    100
comp.os.ms-windows.misc         21        5        74    100
comp.sys.ibm.pc.hardware        22        0        78    100
comp.sys.mac.hardware       

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
print(f"\nPrecision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


Accuracy: 0.7900

Precision: 0.8127
Recall: 0.7900
F1-Score: 0.7785

Classification Report:
                          precision    recall  f1-score   support

             alt.atheism       0.60      0.83      0.70        18
           comp.graphics       0.72      0.72      0.72        18
 comp.os.ms-windows.misc       0.75      0.95      0.84        22
comp.sys.ibm.pc.hardware       0.72      0.84      0.78        25
   comp.sys.mac.hardware       0.88      0.67      0.76        21
          comp.windows.x       1.00      0.28      0.44        25
            misc.forsale       0.78      0.78      0.78        18
               rec.autos       0.81      0.94      0.87        18
         rec.motorcycles       0.82      0.88      0.85        16
      rec.sport.baseball       0.83      0.83      0.83        18
        rec.sport.hockey       0.65      1.00      0.79        15
               sci.crypt       0.70      1.00      0.83        19
         sci.electronics       0.69      0.56    